In [1]:
import numpy as np
import pandas as pd
from os import path, environ
import mrec

импортируем данные

In [2]:
data_dir = "/home/alex_k/Документы/mrec"
environ["data_dir"] = "/home/alex_k/Документы/mrec"

col_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv(
    path.join(data_dir, "lastfm_small.tsv"),
    sep="\t",
    header=None,
    names=col_names
)
data.tail()

,user,artist-mbid,artist-name,total-plays
999995,0e95d8066868c8e189cfe272f690ba11ab47e874,f2eef649-a6d5-4114-afba-e50ab26254d2,sum 41,100
999996,0e95d8066868c8e189cfe272f690ba11ab47e874,5f821136-9935-403c-abb5-e78ddda35e9f,bayside,98
999997,0e95d8066868c8e189cfe272f690ba11ab47e874,90cda7b7-9112-4ec7-9d6f-3f4675ef5130,scary kids scaring kids,93
999998,0e95d8066868c8e189cfe272f690ba11ab47e874,d8354b38-e942-4c89-ba93-29323432abc3,30 seconds to mars,92
999999,0e95d8066868c8e189cfe272f690ba11ab47e874,bc1c8f3f-6f36-4086-a470-87ed813dd429,alexisonfire,91


In [3]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
user           1000000 non-null object
artist-mbid    987168 non-null object
artist-name    1000000 non-null object
total-plays    1000000 non-null int64
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


заполняем пустые ячейки и задаем индексы для "user_id" и "artist_id", нумерацию индексов начинаем с единицы чтобы в дальнейшем использовать mrec

In [4]:
data.fillna("None", inplace=True)
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


разбиваем данные на обучающую и тестовую выборки

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

In [6]:
test_user_set = set(test["user_id"].unique())
train_user_set = set(train["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 1
нет в тестовой выборке, но есть в обучающей: 23
всего пользователей: 20465


In [7]:
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test[test["user_id"].isin(user_ids_to_exclude).values].index
test.drop(bad_indices, inplace=True)
bad_indices = train[train["user_id"].isin(user_ids_to_exclude).values]
train.drop(bad_indices.index, inplace=True)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


запишем в файл обучающую и тестовую выборки (для использования данных файлов mrec_ом для оценки качества предсказаний)

In [8]:
test_file_name = "lastfm.test.0"
test[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, test_file_name),
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"
train[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, train_file_name),
    sep="\t",
    header=False,
    index=False
)

In [9]:
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

создадим зазреженную матрицу, где в качестве строк будет выступать - "artist_id", в качестве столбцов - "user_id", на пересечениях будет стоять информация о количестве прослушиваний "total-plays".

In [10]:
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train["total-plays"].astype(np.double),
    (
        train["artist_id"],
        train["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66800, 20466)
Ненулевых элементов в матрице: 799823
Доля ненулевых элементов: 0.0005850385128306858
Среднее значение ненулевых элементов: 216.62014470701644
Максимальное значение ненулевых элементов: 134993.0
Минимальное значение ненулевых элементов: 1.0


#### метод KNN по косинусной мере

In [11]:
from implicit.nearest_neighbours import CosineRecommender

Для того что бы в дальнейшем попытаться посмотреть на различные метрики, обработаем тестовую выборку. Посчитаем среднее количество прослушиваний у каждого пользователя. Будем исходить из предположения 1 - прослушиваний больше среднего, 0 - прослушиваний меньше среднего, задачу рекомендации будем решать в виде бинарной классификации.

In [12]:
test1=test.groupby('user_id').mean().reset_index()
test1 = test1.drop( ['artist_id'],axis=1)
test1.head()
test2 = test.merge( test1, how = 'outer', left_on = 'user_id', right_on = 'user_id' )
test2.head()

,total-plays_x,user_id,artist_id,total-plays_y
0,110,7590,37535,243.0
1,587,7590,50278,243.0
2,329,7590,17512,243.0
3,227,7590,53744,243.0
4,161,7590,63129,243.0


In [13]:
from sklearn.metrics import mean_squared_error, roc_auc_score, average_precision_score, precision_score
from math import sqrt
# напишем функцию для среднего квадратичного отклонения
def rmse(predict, truth):   
    return sqrt(mean_squared_error(predict, truth))

создадим и обучим нашу модель для различных параметров $K$, оценим качество показаний с помощью различных метрик, в том числе и с помощью mrec

In [26]:
k = [10,20,50,100,200,300,400,500]
for i in k:
    model = CosineRecommender(K=i)
    model.fit(plays)
    user_plays = plays.T.tocsr()
    recommendations = []
    for user_id in test["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
            recommendations.append((user_id, artist_id, score))
    
    with open(path.join(data_dir,"ml-latest-small", test_file_name + ".recs.tsv"), "w") as output_file:
        for user_id in test["user_id"].unique():
            for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
                
    recommendations_data = pd.DataFrame(recommendations, columns=["user_id","artist_id","plays"])
    merged_data = pd.merge(test2, recommendations_data, on=['user_id', 'artist_id'])
    merged_data.rename(columns = {'total-plays_x':'total_plays','total-plays_y':'plays_mean','plays':'plays_predict'}, inplace=True)
    recommend = list(map(lambda x, y: 1 if x > y  else 0, merged_data["plays_predict"], merged_data["plays_mean"]))
    fact = list(map(lambda x, y: 1 if x > y else 0, merged_data["total_plays"], merged_data["plays_mean"]))
    print('K = {},'.format(i), 'ROC-AUC: {}'.format( roc_auc_score(recommend, fact) ))
    print('K = {},'.format(i), 'Precision: {}'.format( precision_score(recommend, fact) ))
    print('K = {},'.format(i), 'Average precision: {}'.format( average_precision_score(recommend, fact) ))
    print('K = {},'.format(i), 'RMSE: {}'.format( rmse(merged_data["plays_predict"], merged_data["total_plays"]) ))
    !./sbin/mrec_evaluate\
    --input_format=tsv --test_input_format=tsv \
    --train $data_dir/lastfm.test.0 \
    --recsdir $data_dir/ml-latest-small
    print()

K = 10, ROC-AUC: 0.4872563862254584
K = 10, Precision: 0.9545867393278837
K = 10, Average precision: 0.9556824319198318
K = 10, RMSE: 958.0601088758082
[2017-12-08 14:20:42,068] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.0354 +/- 0.0000
prec@5         0.0122 +/- 0.0000
prec@10        0.0110 +/- 0.0000
prec@15        0.0073 +/- 0.0000
prec@20        0.0055 +/- 0.0000

K = 20, ROC-AUC: 0.4962373118655933
K = 20, Precision: 0.96767537826685
K = 20, Average precision: 0.9679144292088149
K = 20, RMSE: 1007.1051522979154
[2017-12-08 14:21:31,581] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.0487 +/- 0.0000
prec@5         0.0164 +/- 0.0000
prec@10        0.0144 +/- 0.0000
prec@15        0.0096 +/- 0.0000
prec@20        0.0072 +/- 0.0000

K = 50, ROC-AUC: 0.478137529543879
K = 50, Precision: 0.9745409015025042
K = 50, Average precision: 0.9754811909334437
K = 50, RMSE: 1056.9660267053894
[2017-12-08 14:22:50,702] I

#### метод ALS

In [2]:
import implicit

In [28]:
factors = [10, 50, 100, 200, 300, 400, 500]
regularization = [0.001, 0.01, 0.1, 1.0]

создадим и обучим модель, оценим качество рекомендаций с помощью mrec варьируя количество скрытых факторов и параметр регуляризации

In [40]:
for i in factors:
    for j in regularization:
        model_als = implicit.als.AlternatingLeastSquares(factors=i, regularization=j)
        model_als.fit(plays)
        user_plays = plays.T.tocsr()
        
        with open(path.join(data_dir,"ml-latest-small", test_file_name + ".recs.tsv"), "w") as output_file:
            for user_id in test["user_id"].unique():
                for artist_id, score in model_als.recommend(user_id, user_plays):
                    output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
        print('factors = {},'.format(i), 'regularization = {}'.format(j))       
        !./sbin/mrec_evaluate\
            --input_format=tsv --test_input_format=tsv \
            --train $data_dir/lastfm.test.0 \
            --recsdir $data_dir/ml-latest-small
        print()

factors = 10, regularization = 0.001
[2017-12-08 15:23:16,843] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.2478 +/- 0.0000
prec@5         0.1029 +/- 0.0000
prec@10        0.0846 +/- 0.0000
prec@15        0.0564 +/- 0.0000
prec@20        0.0423 +/- 0.0000



factors = 10, regularization = 0.01
[2017-12-08 15:24:16,558] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.2527 +/- 0.0000
prec@5         0.1037 +/- 0.0000
prec@10        0.0861 +/- 0.0000
prec@15        0.0574 +/- 0.0000
prec@20        0.0431 +/- 0.0000



factors = 10, regularization = 0.1
[2017-12-08 15:25:18,540] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.2542 +/- 0.0000
prec@5         0.1058 +/- 0.0000
prec@10        0.0869 +/- 0.0000
prec@15        0.0579 +/- 0.0000
prec@20        0.0434 +/- 0.0000



factors = 10, regularization = 1.0
[2017-12-08 15:26:19,100] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.2511 +/- 0.0000
prec@5         0.1040 +/- 0.0000
prec@10        0.0861 +/- 0.0000
prec@15        0.0574 +/- 0.0000
prec@20        0.0431 +/- 0.0000



factors = 50, regularization = 0.001
[2017-12-08 15:28:16,984] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.3028 +/- 0.0000
prec@5         0.1295 +/- 0.0000
prec@10        0.1071 +/- 0.0000
prec@15        0.0714 +/- 0.0000
prec@20        0.0535 +/- 0.0000


...


factors = 400, regularization = 1.0
[2017-12-08 17:32:29,791] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.3521 +/- 0.0000
prec@5         0.1564 +/- 0.0000
prec@10        0.1248 +/- 0.0000
prec@15        0.0832 +/- 0.0000
prec@20        0.0624 +/- 0.0000



factors = 500, regularization = 0.001
[2017-12-08 17:48:09,173] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.3257 +/- 0.0000
prec@5         0.1416 +/- 0.0000
prec@10        0.1125 +/- 0.0000
prec@15        0.0750 +/- 0.0000
prec@20        0.0562 +/- 0.0000



factors = 500, regularization = 0.01
[2017-12-08 18:03:44,596] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.3269 +/- 0.0000
prec@5         0.1427 +/- 0.0000
prec@10        0.1133 +/- 0.0000
prec@15        0.0756 +/- 0.0000
prec@20        0.0567 +/- 0.0000



factors = 500, regularization = 0.1
[2017-12-08 18:19:22,077] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.3315 +/- 0.0000
prec@5         0.1456 +/- 0.0000
prec@10        0.1157 +/- 0.0000
prec@15        0.0771 +/- 0.0000
prec@20        0.0578 +/- 0.0000



factors = 500, regularization = 1.0
[2017-12-08 18:34:58,323] INFO: processing /home/alex_k/Документы/mrec/lastfm.test.0...
None
mrr            0.3507 +/- 0.0000
prec@5         0.1557 +/- 0.0000
prec@10        0.1248 +/- 0.0000
prec@15        0.0832 +/- 0.0000
prec@20        0.0624 +/- 0.0000

